In [113]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results=requests.get(url)

results.content

soup = BeautifulSoup(results.content, 'html.parser')

data_table=soup.find('table')


<b> Cleaning and adding the postal code, boroughs, and Neighborhood row by row</b>

In [3]:
table_content=[]
for row in data_table.findAll('td'):
    cell={}
    if row.span.text=="Not assigned":
        pass
    else:
        cell["PostalCode"]=row.p.text[:3]
        cell["Borough"]=(row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_content.append(cell)
print(table_content)

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

<b>This is to see if there are any neighborhoods that have a not assigned value to them</b>

In [5]:
print(df[df["Neighborhood"]=="Not assigned"])
df[df["Neighborhood"]==None]

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


,PostalCode,Borough,Neighborhood


<b> I added the last part as it was in the hint, so I figured it was important to change and replace those values in the Borough section</b>

In [ ]:
df=pd.DataFrame(table_content)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

In [7]:
df.shape

(103, 3)

<p>This is to read the geospatial coordinates file,using geocoder did not work at all, so that block containing that code was deleted.</p>

In [8]:
location_csv=pd.read_csv('file_location_goes_here')
location_csv

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


<p> This is to insert the geospatial coordinates into the dataframe with the boroughs using an inner join and joining them with the PostalCode which they both share</p>

In [25]:
location_csv=location_csv.rename(columns={'Postal Code':'PostalCode'})
location_csv

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [26]:
result=df.merge(location_csv,on="PostalCode")
result

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h3>Make a new dataframe of just the Toronto boroughs</h3>

In [27]:
result_toronto=result[result["Borough"].str.contains('Toronto')]

In [28]:
result_toronto.reset_index(inplace=True)

In [29]:
result_toronto=result_toronto.iloc[:,1:]
result_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


In [30]:
CLIENT_ID = #enter client ID here 
CLIENT_SECRET = #enter client secret here 
VERSION = '20180605'
LIMIT = 100
radius=500


In [40]:
def get_venues(names,latitudes,longitudes):
    venues_list=[]
    i=1
    for name,lat,lng in zip(names,latitudes,longitudes):
        print(name)
        
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
        
        results=requests.get(url).json()["response"]["groups"][0]["items"]
        
        venues_list.append([(
            name,
            lat,
            lng,
            v["venue"]['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results
        
        ])
        
        
        
        nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        
        nearby_venues.columns=  ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        
        
    return (nearby_venues)

In [41]:
toronto_venues=get_venues(result_toronto["Neighborhood"],
                         result_toronto["Latitude"],
                         result_toronto["Longitude"])


Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [176]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
...,...,...,...,...,...,...,...
1572,Enclave of M4L,43.662744,-79.321558,The Ashbridge Estate,43.664691,-79.321805,Garden
1573,Enclave of M4L,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1574,Enclave of M4L,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1575,Enclave of M4L,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station


In [83]:
len(toronto_venues["Venue Category"].unique())

233

In [84]:
from sklearn.cluster import KMeans
#!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors


In [91]:
pd.set_option('display.max_columns', None)


In [93]:
toronto_encoded=pd.get_dummies(toronto_venues[["Venue Category"]],prefix="",prefix_sep="")
toronto_encoded.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [98]:
toronto_encoded.insert(0,"Neighborhoods",toronto_venues["Neighborhood"])

In [102]:
toronto_grouped=toronto_encoded.groupby("Neighborhoods").mean().reset_index()

In [177]:
toronto_grouped

,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.017544,0.035088,0.000000,0.000000,0.00,0.017544,0.017544,0.000000,0.035088,0.000000,0.017544,0.017544,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.0000,0.000000,0.035088,0.000000,0.000000,0.000000,0.000000,0.017544,0.052632,0.087719,0.000000,0.000000,0.00,0.00000,0.017544,0.000000,0.017544,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.000000,0.000000

In [104]:
toronto_grouped.shape

(39, 234)

In [105]:
from sklearn.cluster import KMeans


<p><b>In order to not make this too complicated the number of clusters we will have will be 5 even though that can be improved upon</b></p>

In [212]:
kclust=5


toronto_grouped_clust=toronto_grouped.drop('Neighborhoods',1)

kmeans=KMeans(n_clusters=kclust,random_state=0).fit(toronto_grouped_clust)

kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [213]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0])

In [252]:
def return_most_common_venues(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [253]:
num_top_venues=10

columns=["Neighborhoods"]

for ind in np.arange(num_top_venues):
    columns.append('#{} Most common Venues'.format(ind+1))

    
neighborhoods_venues_sorted=pd.DataFrame(columns=columns)
neighborhoods_venues_sorted["Neighborhoods"]=toronto_grouped["Neighborhoods"]

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:]=return_most_common_venues(toronto_grouped.iloc[ind,:],num_top_venues)


In [254]:
neighborhoods_venues_sorted.insert(0,column="Label",value=kmeans.labels_)
neighborhoods_venues_sorted

,Label,Neighborhoods,#1 Most common Venues,#2 Most common Venues,#3 Most common Venues,#4 Most common Venues,#5 Most common Venues,#6 Most common Venues,#7 Most common Venues,#8 Most common Venues,#9 Most common Venues,#10 Most common Venues
0,0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Farmers Market,Bakery,Seafood Restaurant,Pharmacy,Cheese Shop,Park
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Furniture / Home Store,Burrito Place,Stadium,Restaurant,Italian Restaurant,Intersection,Bar
2,0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Airport,Boutique,Bar,Coffee Shop,Plane,Boat or Ferry,Rental Car Location,Sculpture Garden,Harbor / Marina
3,0,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,New American Restaurant,Juice Bar,Korean Restaurant
4,0,Christie,Grocery Store,Café,Park,Nightclub,Baby Store,Restaurant,Athletics & Sports,Italian Restaurant,Candy Store,Coffee Shop
5,0,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Hotel,Pub,Fast Food Restaurant,Yoga Studio,Gay Bar,Mediterranean Restaurant
6,0,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Gym,Seafood Restaurant,Japanese Restaurant,American Restaurant,Deli / Bodega
7,0,Davisville,Dessert Shop,Sandwich Place,Gym,Italian Restaurant,Thai Restaurant,Sushi Restaurant,Pizza Place,Coffee Shop,Café,Pharmacy
8,0,Davisville North,Pizza Place,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Department Store,Playground,Park,Eastern European Restaurant,Dumpling Restaurant
9,0,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Café,Bank,Bar,Supermarket,Middle Eastern Restaurant,Music Venue,Grocery Store,Brewery


In [217]:
toronto_merged=result_toronto
toronto_merged=toronto_merged.rename(columns={"Neighborhood":"Neighborhoods"})
toronto_merged

,PostalCode,Borough,Neighborhoods,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


In [218]:
toronto_merged=toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'),on="Neighborhoods")
toronto_merged

,PostalCode,Borough,Neighborhoods,Latitude,Longitude,Label,#1 Most common Venues,#2 Most common Venues,#3 Most common Venues,#4 Most common Venues,#5 Most common Venues,#6 Most common Venues,#7 Most common Venues,#8 Most common Venues,#9 Most common Venues,#10 Most common Venues
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Café,Park,Breakfast Spot,Theater,Yoga Studio,Event Space,Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Pizza Place,Electronics Store,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Gastropub,Cosmetics Shop,Cocktail Bar,Beer Bar,Clothing Store,Gym,Park,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Neighborhood,Health Food Store,Pub,Trail,Electronics Store,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Department Store,Donut Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Farmers Market,Bakery,Seafood Restaurant,Pharmacy,Cheese Shop,Park
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,New American Restaurant,Juice Bar,Korean Restaurant
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Nightclub,Baby Store,Restaurant,Athletics & Sports,Italian Restaurant,Candy Store,Coffee Shop
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Deli / Bodega,Gym,Clothing Store,Thai Restaurant,Concert Hall,Salad Place,Hotel
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Bakery,Pharmacy,Café,Bank,Bar,Supermarket,Middle Eastern Restaurant,Music Venue,Grocery Store,Brewery
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,1,Park,Convenience Store,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


<p>I made the 4th cluster a marker instead of a circle marker because it was hard to spot and so a marker made it easier to see</p>

In [227]:
Map_clusters=folium.Map(location=[43.651070,-79.347015],zoom_start=11)


colors_array = cm.rainbow(np.linspace(0, 1, 4))

print(colors_array)
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)



markers_colors=[]
for lat,lon,poi,cluster in zip(toronto_merged["Latitude"],toronto_merged["Longitude"],toronto_merged["Neighborhoods"],toronto_merged["Label"]):
    label=folium.Popup(str(poi)+' Cluster '+str(cluster),parse_html=True)
    if (cluster!=4):
        folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Map_clusters)
    else:
        folium.Marker(
            [lat,lon],
            popup=label,
            icon=folium.Icon(color="orange")
        ).add_to(Map_clusters)
    
Map_clusters


[[5.00000000e-01 0.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.66666667e-01 8.66025404e-01 8.66025404e-01 1.00000000e+00]
 [8.33333333e-01 8.66025404e-01 5.00000000e-01 1.00000000e+00]
 [1.00000000e+00 1.22464680e-16 6.12323400e-17 1.00000000e+00]]
['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


<h4>All cluster 0 neighborhoods</h4>

<p>Cluster 0 seems to have things such as cafe's and coffee shops for the most part among its first 3 most common venues.Also only this cluster also has things such as Light rail station in its top 10 compared to the other clusters. I would for the most part say that cluster 0 is the cofee, and desert cluster with a few odd ones.</p>

In [249]:
toronto_merged.loc[toronto_merged['Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,#1 Most common Venues,#2 Most common Venues,#3 Most common Venues,#4 Most common Venues,#5 Most common Venues,#6 Most common Venues,#7 Most common Venues,#8 Most common Venues,#9 Most common Venues,#10 Most common Venues
0,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Café,Park,Breakfast Spot,Theater,Yoga Studio,Event Space,Restaurant
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Pizza Place,Electronics Store,Italian Restaurant
2,Downtown Toronto,0,Café,Coffee Shop,Gastropub,Cosmetics Shop,Cocktail Bar,Beer Bar,Clothing Store,Gym,Park,Italian Restaurant
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Farmers Market,Bakery,Seafood Restaurant,Pharmacy,Cheese Shop,Park
5,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,New American Restaurant,Juice Bar,Korean Restaurant
6,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Baby Store,Restaurant,Athletics & Sports,Italian Restaurant,Candy Store,Coffee Shop
7,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Deli / Bodega,Gym,Clothing Store,Thai Restaurant,Concert Hall,Salad Place,Hotel
8,West Toronto,0,Bakery,Pharmacy,Café,Bank,Bar,Supermarket,Middle Eastern Restaurant,Music Venue,Grocery Store,Brewery
10,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Brewery,Fried Chicken Joint,Italian Restaurant,Scenic Lookout,Sporting Goods Shop,Restaurant
11,West Toronto,0,Bar,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Café,Men's Store,Asian Restaurant,Cuban Restaurant,Boutique


<h4>All cluster 1 Neighborhoods</h4>

<p> I would consider cluster 1 to be places where physical movement is involved. it contains things such as apark, trail, yoga studio, and the later potion of their common venues seem to be restaurants and the elctronic store. So physical activity  and Food.</p>

In [248]:
toronto_merged.loc[toronto_merged['Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,#1 Most common Venues,#2 Most common Venues,#3 Most common Venues,#4 Most common Venues,#5 Most common Venues,#6 Most common Venues,#7 Most common Venues,#8 Most common Venues,#9 Most common Venues,#10 Most common Venues
9,East York/East Toronto,1,Park,Convenience Store,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
33,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


<h4>All cluster 2 Neighborhoods</h4>


<p>For cluster 2 i would consider this different from the other clusters because it has the health food store as its number 2 most common venues and it also has the neighborhood as its most common venues. I would say this might be the sightseeing cluster along with the health cluster</p> 

In [244]:
toronto_merged.loc[toronto_merged['Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,#1 Most common Venues,#2 Most common Venues,#3 Most common Venues,#4 Most common Venues,#5 Most common Venues,#6 Most common Venues,#7 Most common Venues,#8 Most common Venues,#9 Most common Venues,#10 Most common Venues
3,East Toronto,2,Neighborhood,Health Food Store,Pub,Trail,Electronics Store,Escape Room,Eastern European Restaurant,Dumpling Restaurant,Department Store,Donut Shop


<h4>All cluster 3 Neighborhoods</h4>

<p>I would consider this to be the physical activity but with a much larger emphasis on dining and food than cluster 1. That is probably what differentiated it</p>

In [245]:
toronto_merged.loc[toronto_merged['Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,#1 Most common Venues,#2 Most common Venues,#3 Most common Venues,#4 Most common Venues,#5 Most common Venues,#6 Most common Venues,#7 Most common Venues,#8 Most common Venues,#9 Most common Venues,#10 Most common Venues
29,Central Toronto,3,Playground,Restaurant,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


<h4>All cluster 4 neighborhoods</h4>


<p>It has the same most common venues as cluster 1 but i think this ended up being different because of the fact that it has swim school and it has less restaurants. this might be the physical activity with + swimming section</p>

In [246]:
toronto_merged.loc[toronto_merged['Label'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,#1 Most common Venues,#2 Most common Venues,#3 Most common Venues,#4 Most common Venues,#5 Most common Venues,#6 Most common Venues,#7 Most common Venues,#8 Most common Venues,#9 Most common Venues,#10 Most common Venues
18,Central Toronto,4,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
